<a href="https://colab.research.google.com/github/spaunkoska22/Hate-Speech-Detection-with-ML-NLP/blob/main/Timski_Pretrained_Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing necessary libraries**

In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.3 MB/s eta 0:00:

**Loading Hate Speech and Offensive Language Dataset**

In [2]:
import pandas as pd
import numpy as np

# Load dataset into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/labeled_data.csv')

In [3]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


**Data preparation**

In [5]:
df_bert = df.copy()
df_bert.drop(['Unnamed: 0', 'count','hate_speech', 'offensive_language', 'neither'], axis=1, inplace=True)

In [6]:
df_bert.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [7]:
df_bert["class"].unique()

array([2, 1, 0])


*   Hate speech --> 0
*   Offensive --> 1
*   Neither ---> 2

**Balancing the dataset**

In [8]:
df_bert.pivot_table(columns=['class'], aggfunc='size')

class
0     1430
1    19190
2     4163
dtype: int64

In [9]:
from imblearn.over_sampling import RandomOverSampler

X = df_bert.drop('class', axis=1)
y = df_bert['class']

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

unique, counts = np.unique(y_resampled, return_counts=True)
print(dict(zip(unique, counts)))

{0: 19190, 1: 19190, 2: 19190}


**Train-test split**

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X_resampled,y_resampled,test_size=0.2)

**Making two different datasets:**

*  **Train dataset**
*  **Test dataset**

In [11]:
df_bert_train = pd.concat([X_train,Y_train],axis=1)
df_bert_test= pd.concat([X_test,Y_test],axis=1)

In [12]:
df_bert_train

,tweet,class
24765,you hoe spice,1
34850,Stoni is a fuckin queer...,0
35541,@UberFacts studies show that you're a faggot,0
1612,&#8220;@Xenoss_: ''single af'' we don't care y...,1
24621,u can tell yo bitch we some yung rich homies,1
...,...,...
23957,http://t.co/bPU86qzYFt Crystal Star &#8211; Le...,1
39578,"@teenagesleuth .we got 1, except eric holder g...",0
32605,Jai may make an appearance at the party... lov...,0
21489,"The Devil Wears Charlie, by Revlon. #lesserfilms",2


In [13]:
df_bert_test

,tweet,class
26069,@sassyharryballs ur ballsack is hairy gtfo tra...,0
8350,Can't no hoe #FixMyLife,1
42165,"bitch kill yoself, go on to the bathroom and e...",0
26630,I like taking niggas hoes but I stop doing tha...,0
17514,RT @TAXSTONE: Y'all bitches sending inviatatio...,1
...,...,...
7843,At the Mexican laundry mat and it's PACKED den...,1
8222,Buddy slapped his rent stress out on that mess...,1
36991,@Nien_Nunb what did you search? Gay redneck ep...,0
26680,@PrayForJack new york niggas quick to take off...,0


**Renaming the columns of the datasets**

In [14]:
df_bert_train.columns = ['Tweet','label']
df_bert_test.columns = ['Tweet','label']

In [15]:
df_bert_train.head()

,Tweet,label
24765,you hoe spice,1
34850,Stoni is a fuckin queer...,0
35541,@UberFacts studies show that you're a faggot,0
1612,&#8220;@Xenoss_: ''single af'' we don't care y...,1
24621,u can tell yo bitch we some yung rich homies,1


In [16]:
df_bert_test.head()

,Tweet,label
26069,@sassyharryballs ur ballsack is hairy gtfo tra...,0
8350,Can't no hoe #FixMyLife,1
42165,"bitch kill yoself, go on to the bathroom and e...",0
26630,I like taking niggas hoes but I stop doing tha...,0
17514,RT @TAXSTONE: Y'all bitches sending inviatatio...,1


**Saving the datasets**

In [17]:
df_bert_train.to_csv('train.csv',index=False)
df_bert_test.to_csv('test.csv',index=False)

**Loading pretrained model**

In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

**Loading datasets**

In [19]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c7e46980cbe84319/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

**Encoding 'Tweet' column using tokenizer**

In [20]:
encoded_dataset = dataset.map(lambda t: tokenizer(t['Tweet'],  truncation=True), batched=True,load_from_cache_file=False)
encoded_dataset

Map:   0%|          | 0/46056 [00:00<?, ? examples/s]

Map:   0%|          | 0/11514 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Tweet', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 46056
    })
    test: Dataset({
        features: ['Tweet', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11514
    })
})

**Building model for our dataset**

In [21]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Training our model**

In [22]:
from transformers import TrainingArguments, Trainer

arg = TrainingArguments(
    "sentiment",
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_eval_batch_size=8,
    per_device_train_batch_size=8,
    seed=19
)

trainer = Trainer(
    model=model,
    args=arg,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test']
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.629300
1000,0.461400
1500,0.405100
2000,0.379200
2500,0.311200
3000,0.294300
3500,0.255700
4000,0.227100
4500,0.202300
5000,0.188000


TrainOutput(global_step=5757, training_loss=0.3115375340554332, metrics={'train_runtime': 670.7979, 'train_samples_per_second': 68.659, 'train_steps_per_second': 8.582, 'total_flos': 1284704706969936.0, 'train_loss': 0.3115375340554332, 'epoch': 1.0})

**Predicting values with the model**

In [23]:
y_pred = trainer.predict(encoded_dataset['test'])
y_pred

PredictionOutput(predictions=array([[ 4.5883226 , -0.73556745, -4.702498  ],
       [-2.0961087 ,  4.2333107 , -1.7785263 ],
       [ 4.858006  , -1.3145673 , -4.370892  ],
       ...,
       [ 4.1074715 , -1.1595135 , -3.8127477 ],
       [ 4.78326   , -1.069932  , -4.5371404 ],
       [-3.98563   , -0.614546  ,  5.237226  ]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 0, 2]), metrics={'test_loss': 0.15401965379714966, 'test_runtime': 39.8126, 'test_samples_per_second': 289.205, 'test_steps_per_second': 36.169})

In [24]:
y_pred = y_pred.predictions

In [25]:
y_pred = [np.argmax(y_pred[i]) for i in range(0,len(y_pred))]

**Evaluating our model**

In [26]:
from sklearn.metrics import confusion_matrix, classification_report,f1_score

print(confusion_matrix(Y_test, y_pred))

print(classification_report(Y_test, y_pred))

f1_score(df_bert_test['label'],y_pred,average='macro')

[[3739   31    6]
 [ 207 3605  109]
 [   5   27 3785]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      3776
           1       0.98      0.92      0.95      3921
           2       0.97      0.99      0.98      3817

    accuracy                           0.97     11514
   macro avg       0.97      0.97      0.97     11514
weighted avg       0.97      0.97      0.97     11514



0.9664707113580687